In [15]:
import geojson
import folium
import pandas as pd
import geopandas as gpd
import json

In [16]:
df = pd.read_excel('data.xlsx')

In [17]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LON, df.LAT), crs="EPSG:4326")

In [18]:
estimate_crs = gdf.estimate_utm_crs()
gdf = gdf.to_crs(estimate_crs)
gdf['geometry'] = gdf['geometry'].buffer(2000, 16)
gdf = gdf.to_crs({'init': 'epsg:4326'})

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [19]:
gdf['fill'] = '#ff0000'  # Прозрачная красная заливка
gdf['fill-opacity'] = 0.3  # 50% прозрачность
gdf['outline-style'] = "none"  # Нет обводки
gdf.to_file("output.geojson", driver="GeoJSON")

In [20]:
m = folium.Map(location = [55.750066, 37.614356], zoom_start=10)

folium.GeoJson(
    gdf,
    style_function=lambda feature: {
        "fillColor": "#ff0000",
        "fill-opacity": 0.3,
        "stroke": False
}
    ).add_to(m)
m.save('geo.html')

In [21]:
m = folium.Map(location = [55.750066, 37.614356], zoom_start=10)

for _, r in gdf.iterrows():
    sim_geo = gpd.GeoSeries(gdf['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data = geo_j, style_function=lambda x: {'fillColor': 'orange'})

    geo_j.add_to(m)

    m.save('geo.html')